<a href="https://colab.research.google.com/github/SruthiChilukuri/advanced_deep_learning/blob/master/semi_supervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Semi Supervised Learning using Autoencoders**

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving creditcard.csv to creditcard.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['creditcard.csv']))

In [ ]:
#exploing the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
#lookimg at the distribution of the classes in the data
count_of_class = df['Class'].value_counts().to_frame().reset_index()

In [ ]:
count_of_class

,index,Class
0,0,284315
1,1,492


In [ ]:
#as we can see; this dataset is highly imbalanced with more number of non fradulant cases. This can make the model more biased.
#Lets consider 1000 samples of each class
nfr = df[df['Class'] == 0].sample(1000)
fr = df[df['Class'] == 1]

df = nfr.append(fr).sample(frac=1).reset_index(drop=True)
X = df.drop(['Class'], axis = 1).values
Y = df["Class"].values


In [ ]:
df.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,147501.0,-1.611877,-0.408410,-3.829762,6.249462,-3.360922,1.147964,1.858425,0.474858,-3.838399,-1.445375,2.113900,-3.259702,-0.315347,-1.808103,-0.741877,-2.986920,-4.815863,-1.271655,1.701685,2.425677,1.245582,0.616383,2.251439,-0.066096,0.538710,0.541325,-0.136243,-0.009852,996.27,1
1,120812.0,0.094720,0.897346,-0.313058,0.245447,1.100110,-0.532063,1.327967,-0.113287,-0.586736,-0.231576,0.049824,0.022103,-1.408819,0.794568,-1.529350,-0.375468,-0.555215,0.205845,-0.001250,-0.291944,0.147885,0.490201,-0.285951,-0.424665,0.188195,-0.558016,0.045850,0.017534,15.18,0
2,44447.0,-0.697922,0.676923,1.713175,0.130948,0.204139,-0.416450,1.478250,-0.284246,-0.519367,-0.584856,-0.031981,0.146590,0.160928,-0.034888,0.413163,-0.113670,-0.514918,-0.571761,-1.662260,0.098298,0.234464,0.681076,-0.005150,0.387433,0.062785,-0.507376,-0.077728,-0.089320,110.00,0
3,151514.0,-3.063968,-0.796897,0.200810,0.410748,0.597087,1.469069,-0.339070,1.029111,-0.069509,-0.492955,0.413607,0.504536,-0.415712,0.633279,0.774931,0.254013,-0.254066,-0.064219,-1.281403,-0.734714,0.312957,0.978376,-1.051702,-1.562756,-0.289341,-0.501238,-0.263225,-0.572841,150.00,0
4,102572.0,-28.709229,22.057729,-27.855811,11.845013,-18.983813,6.474115,-43.557242,-41.044261,-13.320155,-24.588262,3.481952,-9.128341,-3.008255,0.796580,-2.102299,-5.660366,-13.032785,-5.392349,-0.901041,11.059004,-22.797604,8.316275,5.466230,0.023854,-1.527145,-0.145225,-5.682338,-0.439134,0.01,1
5,79811.0,-3.702823,-2.114190,0.490765,2.041532,2.136139,-0.885896,-0.294598,-0.103287,-0.094988,1.383355,1.426631,0.674911,0.464164,0.180167,1.512018,-0.494356,-0.101468,0.236752,1.768233,-1.358833,-0.452859,0.949248,1.367029,-0.228609,0.161950,-0.034948,0.738952,0.484796,16.62,0
6,118832.0,2.162760,-1.238885,0.063662,-0.912555,-1.480984,0.013461,-1.675211,0.127731,0.713710,0.689094,-1.569801,-0.777109,0.386043,-0.895883,0.694106,1.882825,-0.365570,-0.419707,0.463375,0.004118,0.375718,1.185962,0.102516,-0.684163,-0.324838,-0.046454,0.054129,-0.038738,11.99,0
7,93920.0,-12.381048,8.213022,-16.962530,7.116091,-9.772826,-3.666836,-16.147363,2.078706,-4.250657,-16.746044,7.425801,-15.564838,-0.426338,-14.029538,-1.681889,-11.133761,-15.833589,-5.748533,2.271082,0.537795,0.167703,1.503413,-0.767755,0.371951,-1.415639,-0.517022,-0.434621,0.292721,97.00,1
8,120276.0,2.000590,-1.324189,-1.676207,-1.167854,-0.179043,0.274783,-0.624379,0.014384,-0.312021,0.894985,-0.194797,-0.485356,-0.264924,0.011132,-0.571655,1.468368,-0.384791,-0.508154,1.323177,0.251499,0.232241,0.359000,-0.070933,-0.394062,0.020437,-0.156837,-0.053822,-0.050643,136.70,0
9,83716.0,0.873260,-0.965394,0.835950,0.196431,-1.180752,0.168709,-0.567718,0.198636,1.004543,-0.348419,0.552570,0.969431,-0.390085,-0.268095,-0.919340,0.209844,-0.198678,0.040710,0.828110,0.255077,-0.108112,-0.476197,-0.113641,0.075769,0.076758,0.929281,-0.071364,0.028656,173.05,0


In [ ]:
#To perform semi supervised learning; we can use autoencoders. These are special type of neural networks that can learn the lower levels of representations.
input = Input(shape=(X.shape[1],))
encoded = Dense(100, activation = 'tanh')(input)
encoded = Dense(50, activation='relu')(encoded)
decode = Dense(50, activation='tanh')(encoded)
decode = Dense(100, activation='tanh')(decode)
output = Dense(X.shape[1], activation='relu')(decode)

In [ ]:
autoencoder = Model(input, output)
autoencoder.compile(optimizer="adadelta", loss="mse")

In [ ]:
#scaling the features before running the model; using the MinMax scaler
y = df["Class"].values
x = df.drop(["Class"], axis =1)

x_scaled = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scaled[y==0], x_scaled[y==1]


In [ ]:
autoencoder.fit(x_norm[0:2000], x_norm[0:2000], 
                batch_size = 256, epochs = 100, 
                shuffle = True, validation_split = 0.25)

Epoch 1/100
3/3 [==============================] - 0s 18ms/step - loss: 0.3338 - val_loss: 0.3351
Epoch 2/100
3/3 [==============================] - 0s 8ms/step - loss: 0.3336 - val_loss: 0.3349
Epoch 3/100
3/3 [==============================] - 0s 8ms/step - loss: 0.3334 - val_loss: 0.3347
Epoch 4/100
3/3 [==============================] - 0s 8ms/step - loss: 0.3332 - val_loss: 0.3346
Epoch 5/100
3/3 [==============================] - 0s 8ms/step - loss: 0.3330 - val_loss: 0.3344
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 0.3328 - val_loss: 0.3342
Epoch 7/100
3/3 [==============================] - 0s 8ms/step - loss: 0.3326 - val_loss: 0.3340
Epoch 8/100
3/3 [==============================] - 0s 9ms/step - loss: 0.3324 - val_loss: 0.3338
Epoch 9/100
3/3 [==============================] - 0s 8ms/step - loss: 0.3322 - val_loss: 0.3336
Epoch 10/100
3/3 [==============================] - 0s 8ms/step - loss: 0.3320 - val_loss: 0.3334
Epoch 11/100
3/3 [==========

In [ ]:
#train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))
print("")


Classification Report: 
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.97       255
         1.0       1.00      0.85      0.92       118

    accuracy                           0.95       373
   macro avg       0.97      0.92      0.94       373
weighted avg       0.95      0.95      0.95       373

Accuracy Score:  0.9517426273458445

